In [1]:
import h5py
#import pylab as pl
import numpy as np
#import matplotlib as mpl
import networkx
import ipyvolume as ipv

In [2]:
def split_unique_id(unique_id):
    """Splits the ids assign to the subhalos by the merger tree code by snap number and subfind number """
    subfind_number = int(unique_id % 1e6)
    snap_number = int((unique_id - subfind_number) / 1e6)
    
    return snap_number, subfind_number

def get_main_branch_unique_ids(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    i = 0
    while True:
        succesors = list(subtree.successors(node))
        if len(succesors) == 0:
            break
        node = succesors[0] # select only the first succesor (main branch)
        mpb.append(node)
        
    return mpb

def get_main_branch_and_progNumb(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    numb = [[node], ]
    i = 0
    while True:
        successors = list(subtree.successors(node))
        if len(successors) == 0:
            break
        node = successors[0] # select only the first succesor (main branch)
        mpb.append(node)
        numb.append(successors)
        
    return mpb, numb


def set_view(figure, framenr, fraction):
    ipv.view(fraction*360, 0.)
    
def set_angles(fig, i, fraction):
    fig.angley = fraction*np.pi*2
    
# a bunch of constants
UnitMass_in_g = 1.989e+43
UnitTime_in_s = 3.08568e+16
UnitVelocity_in_cm_per_s = 100000
UnitDensity_in_cgs = 6.7699e-22
UnitEnergy_in_cgs = 1.989e+53
GAMMA_MINUS1 = 2./3.
PROTONMASS = 1.6726e-24
BOLTZMANN = 1.3806e-16    

def tempFromMass(Mass, Abund, IE, ne1):
    XH = Abund[:,6]/Mass
    yHelium = (1. - XH)/(4.*XH)
    mu = (1 + 4.* yHelium)/ (1.+ yHelium + ne1 )
    
    temp = GAMMA_MINUS1 * IE * mu * 1.6726 / 1.3806 * 1.e-8 # / BOLTZMANN  * PROTONMASS
    temp = temp * 1e10 #   UnitEnergy_in_cgs / UnitMass_in_g;
    return temp

In [3]:
sim = h5py.File('/data/cielo/simulations/LG1/LG1.hdf5', 'r')
trees = networkx.read_multiline_adjlist('/data/cielo/simulations/LG1/LG1_tree.dat')

In [4]:
def plotItIn3D(fofID, centralID):
    uniqueID = int(127*1e6+fofID)
    stree = networkx.dfs_tree(trees, str(uniqueID))
    mtree, numbs = get_main_branch_and_progNumb(stree, str(uniqueID))
    
    uniqueC = int(127*1e6+centralID)
    streeC = networkx.dfs_tree(trees, str(uniqueC))
    mtreeC = get_main_branch_unique_ids(streeC, str(uniqueC))
    
    # find all the particleIDs that once belong to the subgroup
#     Idlist, zlist, zcen = [], [], []
    Idlist, templist = [], [] # that's it.

    galH, cenH = [], []
    
    for mm, mmC in zip(mtree[1:], mtreeC[1:]): # ignore the first one
        snap, idd = split_unique_id(int(mm))
        snapC, iddC = split_unique_id(int(mmC))
        tempOff = sim['SnapNumber_{}/SubGroups/PartType0/Offsets'.format(snap)][idd].astype('int')
        pgal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][idd]
#         zt = sim['SnapNumber_{}/Header/Redshift'.format(snap)][()]
        if snapC==snap:
            pcen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snapC)][iddC]
        galH.append(pgal)
        cenH.append(pcen)
        if tempOff[0]>= 0 and tempOff[1]>=0:
            tempIds = sim['SnapNumber_{}/PartType0/ParticleIDs'.format(snap)][tempOff[0]:tempOff[1]]
            IE = sim['SnapNumber_{}/PartType0/InternalEnergy'.format(snap)][tempOff[0]:tempOff[1]]
            Mass = sim['SnapNumber_{}/PartType0/Masses'.format(snap)][tempOff[0]:tempOff[1]]
            Abund = sim['SnapNumber_{}/PartType0/Abundances'.format(snap)][tempOff[0]:tempOff[1]]
            ne1 = sim['SnapNumber_{}/PartType0/ElectronAbundance'.format(snap)][tempOff[0]:tempOff[1]]
            tempT = tempFromMass(Mass, Abund, IE, ne1)
            #tempT = sim['SnapNumber_{}/PartType0/InternalEnergy'.format(snap)][tempOff[0]:tempOff[1]]
            Idlist.extend(list(tempIds))
            templist.extend(list(tempT))
#             zlist.extend([zt]*tempIds.size)

#     Idlist, zlist = np.array(Idlist), np.array(zlist)
    Idlist, templist = np.array(Idlist), np.array(templist)
    Idlist, Idunique = np.unique(Idlist, return_index=True) # remove duplicates
#     zlist = zlist[Idunique]
    templist = templist[Idunique]


    galH = np.array(galH)
    cenH = np.array(cenH)
#     zcen = np.array(zcen)
    
    # the total list of particles that once belong to a give subgroup, lest see where are today.
    TdayOff = sim['SnapNumber_127/SubGroups/PartType0/Offsets'][fofID].astype('int')
    TdayCoord = sim['SnapNumber_127/PartType0/Coordinates'][TdayOff[0]:TdayOff[1]]

    TdayTotalIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][()]
    
    IndexToday = np.in1d(TdayTotalIDs, Idlist)
    IndexToday_ = np.where(IndexToday)[0]
    
    #remove the ones that are not part of the group today.
    TdayIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][TdayOff[0]:TdayOff[1]]

    IndexNot = np.in1d(Idlist, TdayIDs)
    IdNot = Idlist[np.where(IndexNot==False)[0]]

    IndexT = np.in1d(TdayTotalIDs, IdNot)
    IndexT_ = np.where(IndexT)[0]
    LostCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_]

    # ... and the redshift
    SurvIds_ = TdayTotalIDs[IndexT_]
    iidex = np.in1d(Idlist, SurvIds_)
    iidex_ = np.where(iidex)[0]
#     zlist_, Idlist_ = zlist[iidex_], Idlist[iidex_]
    templist_, Idlist_ = templist[iidex_], Idlist[iidex_]

    is1 = np.argsort(SurvIds_) # the target
    is2 = np.argsort(Idlist_)

    temporder = np.zeros(IndexT_.size)
    temporder[is1] = templist_[is2]
    
    #now lets cut at  a certain distance
    distance = 600 # distance in ckpc

    posGal = sim['SnapNumber_127/SubGroups/SubGroupPos'][fofID]
    posCen = sim['SnapNumber_127/SubGroups/SubGroupPos'][centralID]

    dx, dy, dz = LostCoord[:,0]-posGal[0], LostCoord[:,1]-posGal[1], LostCoord[:,2]-posGal[2]
    distToC = np.sqrt(dx**2+dy**2+dz**2)

    icut = np.where(distToC<distance)[0]
    IndexT_near = IndexT_[icut]
#     zorder_near = zorder[icut]
#    temporder_near = temporder[icut]
    IndexT_near = IndexT_
#     zorder_near = zorder
    temporder_near = temporder
    
#     icut = np.where(distToC<distance)[0]
#     #IndexT_near = IndexT_[icut]
#     #zorder_near = zorder[icut]
#     IndexT_near = IndexT_
#     zorder_near = zorder
    
    Index_limited = IndexT_[icut]
    
    NearCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_near]
    NearestCoord = sim['SnapNumber_127/PartType0/Coordinates'][Index_limited]
        
    # ok, lest see velocity and stuff
    velCen = sim['SnapNumber_128/SubGroups/SubGroupVel'][centralID]
    velSat = sim['SnapNumber_128/SubGroups/SubGroupVel'][fofID]

    cenRep = posGal - posCen

    velRep = velSat - velCen
    
    xo, yo, zo = NearCoord[:,0], NearCoord[:,1], NearCoord[:,2]
    xoC, yoC, zoC = xo - posCen[0], yo - posCen[1], zo - posCen[2]
    velX, velY, velZ = velRep[0], velRep[1], velRep[2]

    xoT, yoT, zoT = NearestCoord[:,0] - posCen[0], NearestCoord[:,1] - posCen[1], NearestCoord[:,2] - posCen[2]
    
    #cenPx, cenPy, cenPz = galH[:,0] - posCen[0], galH[:,1] - posCen[1], galH[:,2] - posCen[2]
    cenPx, cenPy, cenPz = galH[:,0] - cenH[:,0], galH[:,1] - cenH[:,1], galH[:,2] - cenH[:,2]

    nzbins = 4
    # zlim = np.linspace(zorder_near.min(), zorder_near.max(), nzbins+1)
    tli = np.percentile(temporder_near, [0, 25, 50, 75, 100])
    tlim = np.array(tli)

    for i in range(tlim.size):
        tlim[i] = float(int(tlim[i]*10))/10
    
    colourlist = ['blue', 'green', 'orange', 'red']
    
    fig = ipv.figure(width=750, height=750)
    
    xmin, xmax = xoT.min(), xoT.max()
    ymin, ymax = yoT.min(), yoT.max()
    zmin, zmax = zoT.min(), zoT.max()
    
    for ti, tm, colour in zip(tlim[:-1], tlim[1:], colourlist):
        #print(colour,'\t:','{} < Internal Energy [km^2/s^2] < {}'.format(ti, tm))
        print(colour,'\t:','{} < Temperature/K < {}'.format(ti, tm))
        i = np.where((temporder_near>ti) & (temporder_near<tm))
        xoCT ,yoCT, zoCT = xoC[i], yoC[i], zoC[i]
        scatter = ipv.scatter(xoCT, yoCT, zoCT, marker='sphere', size=.25, color=colour)
        
#         line = ipv.plot(cenPx[iz], cenPy[iz], cenPz[iz], ls='--', color=colour)
#     scatter = ipv.scatter(xoC, yoC, zoC, marker='sphere', size=.25, color=colname)
#     line = ipv.plot(cenPx[iz], cenPy[iz], cenPz[iz], ls='--', color=colname2)
    
    ipv.plot(cenPx, cenPy, cenPz, ls='--', color='black')
    
    quiv = ipv.quiver(np.array([cenRep[0]]), np.array([cenRep[1]]), np.array([cenRep[2]]), np.array([velX]), np.array([velY]), np.array([velZ]), color='black', size=10)
    
    #ipv.xlabel('x/(ckpc h^{-1})')
    #ipv.ylabel('y/(ckpc h^{-1})')
    #ipv.zlabel('z/(ckpc h^{-1})')
    
#     for i, z in enumerate(zcen):
#         if len(numbs[i]) > 1:
#             main = numbs[i][0]
#             snap, fofi = split_unique_id(int(main))
#             offM = sim['SnapNumber_{}/SubGroups/PartType1/Offsets'.format(snap)][fofi]
#             mainN = offM[1]-offM[0]
#             otherN = 0
#             for n in numbs[i][1:]:
#                 snapS, fofS = split_unique_id(int(n))
#                 offS = sim['SnapNumber_{}/SubGroups/PartType1/Offsets'.format(snapS)][fofS]
#                 otherN += offS[1] - offS[0]
#             ratio = otherN/mainN
#             mergers = ipv.scatter(np.array([cenPx[i]]), np.array([cenPy[i]]), np.array([cenPz[i]]), marker='sphere', size=ratio*20, color='black')
    
#     ipv.movie('{}.gif'.format(fofID), set_angles, fps=20, frames=20*4, endpoint=False)

    mmin = np.min([xmin, ymin, zmin])
    mmax = np.max([xmax, ymax, zmax])

    ipv.xlim(mmin, mmax)
    ipv.ylim(mmin, mmax)
    ipv.zlim(mmin, mmax)
    
    ipv.show()

In [5]:
# 4338, 4341 (4337), 4470 y 4474 (4469)
plotItIn3D(4338,4337)

blue 	: 1634.7 < Temperature/K < 9982.3
green 	: 9982.3 < Temperature/K < 10100.2
orange 	: 10100.2 < Temperature/K < 10544.0
red 	: 10544.0 < Temperature/K < 491102.2


In [6]:
# filename = '4338_Temperature.gif'
# ipv.movie(filename, set_angles, fps=20, frames=20*4, endpoint=False)

In [7]:
plotItIn3D(4341,4337)

blue 	: 3439.7 < Temperature/K < 9981.1
green 	: 9981.1 < Temperature/K < 10043.9
orange 	: 10043.9 < Temperature/K < 10486.6
red 	: 10486.6 < Temperature/K < 253389.8


In [8]:
plotItIn3D(4470,4469)

blue 	: 1012.3 < Temperature/K < 10097.0
green 	: 10097.0 < Temperature/K < 11304.8
orange 	: 11304.8 < Temperature/K < 142095.9
red 	: 142095.9 < Temperature/K < 2127898.7


In [9]:
plotItIn3D(4474,4469)

blue 	: 1546.2 < Temperature/K < 9985.1
green 	: 9985.1 < Temperature/K < 10139.3
orange 	: 10139.3 < Temperature/K < 10783.2
red 	: 10783.2 < Temperature/K < 583155.0
